In [ ]:
from pathlib import Path
import re
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt
from lumicks import pylake
from enum import Enum

In [ ]:
data_dirs = [Path("/Users/sylvi/optical_data/loading_markers/data")]

for data_dir in data_dirs:
    assert data_dir.exists(), f"Data directory {data_dir} does not exist."

output_folder = Path("/Users/sylvi/optical_data/loading_markers/processed/")

markers = {}

for data_dir in data_dirs:
    for file in data_dir.glob("*.h5"):
        if file.is_file() and "Marker" in file.name:
            print(f"Loading file {file}")
            marker_data = pylake.File(file)
            print(type(marker_data))

            # get metadata from the file name
            tel_reps = re.search(r"Tel(\d+)", file.name)
            if tel_reps:
                tel_reps = int(tel_reps.group(1))
            else:
                raise ValueError(f"Could not find telereps in file name {file.name}")

            protein_name = re.search(r" (\w+)(?= Marker \d+)", file.name)
            if protein_name:
                protein_name = protein_name.group(1)
            else:
                raise ValueError(f"Could not find protein name in file name {file.name}")

            # extract the curves
            for curve_id, curve_data in marker_data.fdcurves.items():
                print(f"Curve ID: {curve_id}")
                force_data = curve_data.f.data
                curve_data.plot_scatter()
                plt.title(f"Force-distance plot from built-in plot_scatter() method")
                plt.show()
                print(curve_data.f)
                plt.plot(force_data)
                plt.title(f"Force data retrieved via file.fdcurves[{curve_id}].f.data")
                plt.show()

                distance_data = curve_data.d.data
                plt.plot(distance_data)
                plt.title(f"Distance data retrieved via file.fdcurves[{curve_id}].d.data")
                plt.show()

                plt.plot(distance_data, force_data)
                plt.title(f"Force vs Distance for curve {curve_id}")
                plt.xlabel("Distance (nm)")
                plt.ylabel("Force (pN)")
                plt.show()

                plt.scatter(distance_data, force_data, s=8)
                plt.title(f"Force vs Distance Scatter for curve {curve_id}")
                plt.xlabel("Distance (nm)")
                plt.ylabel("Force (pN)")
                plt.show()

                distance_diffs = np.diff(distance_data)
                plt.plot(distance_diffs)
                plt.title(f"Distance differences for curve {curve_id}")
                plt.xlabel("Index")
                plt.ylabel("Distance difference (nm)")
                plt.show()

                distance_diff_direction_threshold = 0.02
                distance_increasing = distance_diffs > distance_diff_direction_threshold
                distance_decreasing = distance_diffs < -distance_diff_direction_threshold
                distance_stable = np.abs(distance_diffs) <= distance_diff_direction_threshold
                distance_state = np.zeros_like(distance_diffs)
                distance_state[distance_increasing] = 1
                distance_state[distance_decreasing] = -1
                plt.scatter(range(len(distance_state)), distance_state, s=8)
                plt.title(f"Distance state for curve {curve_id}")
                plt.xlabel("Index")
                plt.ylabel("State (1: increasing, -1: decreasing, 0: stable)")
                plt.show()

                oscillations = []
                current_oscillations = {
                    "increasing": {
                        "force": None,
                        "distance": None,
                    },
                    "decreasing": {
                        "force": None,
                        "distance": None,
                    },
                    "stable": {
                        "force": None,
                        "distance": None,
                    },
                }
                previous_stop_index = 0
                current_state = distance_state[0]
                assert current_state == 1, "Initial state should be increasing"

                for new_index, new_state in enumerate(distance_state):
                    if new_state == current_state:
                        continue
                    else:
                        print(f"new state: {new_state}, current state: {current_state}, new index: {new_index}")
                        # end of current state, add array segment, inclusive of start and exclusive of end
                        current_segment_force = force_data[previous_stop_index:new_index]
                        current_segment_distance = distance_data[previous_stop_index:new_index]
                        if new_state == 1:
                            # started a new increasing segment, so save the previous oscillation and reset
                            assert current_state == 0, "Current state should be stable before increasing"
                            current_oscillations["stable"]["force"] = current_segment_force
                            current_oscillations["stable"]["distance"] = current_segment_distance
                            current_state = new_state
                            previous_stop_index = new_index
                            # after a stable segment, we append this oscillation to the list and reset
                            # check each force and distance array are not None
                            if (
                                current_oscillations["increasing"]["force"] is None
                                or current_oscillations["increasing"]["distance"] is None
                                or current_oscillations["decreasing"]["force"] is None
                                or current_oscillations["decreasing"]["distance"] is None
                                or current_oscillations["stable"]["force"] is None
                                or current_oscillations["stable"]["distance"] is None
                            ):
                                raise ValueError("One of the oscillation segments is None")
                            oscillations.append(current_oscillations)
                            current_oscillations = {
                                "increasing": {
                                    "force": None,
                                    "distance": None,
                                },
                                "decreasing": {
                                    "force": None,
                                    "distance": None,
                                },
                                "stable": {
                                    "force": None,
                                    "distance": None,
                                },
                            }
                        elif new_state == -1:
                            # started a new decreasing segment
                            assert current_state == 1, "Current state should be increasing before decreasing"
                            current_oscillations["increasing"]["force"] = current_segment_force
                            current_oscillations["increasing"]["distance"] = current_segment_distance
                            current_state = new_state
                            previous_stop_index = new_index
                        elif new_state == 0:
                            # started a new stable segment
                            if current_state == 1:
                                # this could be where we have reached the peak and are waiting for a decrease.
                                # ignore and continue
                                continue
                            else:
                                current_oscillations["decreasing"]["force"] = current_segment_force
                                current_oscillations["decreasing"]["distance"] = current_segment_distance
                                previous_stop_index = new_index
                                current_state = new_state
                        else:
                            raise ValueError(f"Unexpected state value: {new_state}")
                # handle the last segment
                # check if the last segment is stable
                if current_state == 0:
                    current_segment_force = force_data[previous_stop_index : new_index + 1]
                    current_segment_distance = distance_data[previous_stop_index : new_index + 1]
                    current_oscillations["stable"]["force"] = current_segment_force
                    current_oscillations["stable"]["distance"] = current_segment_distance
                    oscillations.append(current_oscillations)
                else:
                    raise ValueError(f"Last segment is not stable, state: {current_state}")

                break
            break